In [1]:
# Gerekli kütüphanelerin içe aktarılması
import yfinance as yf  # Finansal veri indirmek için
import numpy as np  # Sayısal işlemler için
from sklearn.preprocessing import MinMaxScaler  # Veri ölçeklendirmek için
from tensorflow.keras.models import Sequential  # Model oluşturmak için
from tensorflow.keras.layers import LSTM, Dense, Dropout , Activation, BatchNormalization # Model katmanları
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau # Model eğitimini durdurmak için
from tensorflow.keras.regularizers import l1, l2

from sklearn.metrics import mean_squared_error  , mean_absolute_error , r2_score  # Hata ölçümü için
import mplfinance as mpf  # Finansal verileri görselleştirmek için
import pandas as pd  # Veri işleme için
import plotly.graph_objects as go

In [2]:
#tensorflow gpu var mı yok mu kontrolü
import tensorflow as tf
tf.test.gpu_device_name()


''

In [3]:
# Veri Toplama
btc_data = yf.download('BTC-USD', period='3600d')  # Bitcoin verilerini yfinance ile indirme
#btc_data

[*********************100%%**********************]  1 of 1 completed


In [4]:
# İndirilen Verileri Mum Çubukları ile Görselleştirme
#mpf.plot(btc_data, type='candle', volume=True, style='yahoo', title='BTC-USD') 

In [5]:
# Veri Hazırlama
btc_df = btc_data[['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume']]  # İlgili sütunların seçilmesi


In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))  # Ölçekleyici tanımlama
scaled_data = scaler.fit_transform(btc_df)  # Verilerin ölçeklendirilmesi


In [7]:

# Zaman serisi veri seti oluşturma fonksiyonu
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step - 1):
        a = dataset[i:(i + time_step), 0:]  # Girdi verileri
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0:])  # Çıktı verileri
    return np.array(dataX), np.array(dataY)

time_step = 7  # Zaman adımı
X, y = create_dataset(scaled_data, time_step)  # X ve y veri setlerinin oluşturulması




In [8]:
# Veri setini eğitim ve test setlerine ayırma
test_size = 300  # Test setinin boyutu
train_size = int(len(X) - test_size)  # Eğitim setinin boyutu
X_train, X_test = X[0:train_size,:,:], X[train_size:len(X),:,:]  # Eğitim ve test girdileri
y_train, y_test = y[0:train_size,:], y[train_size:len(y),:]  # Eğitim ve test çıktıları


In [9]:
units1 = 256  # İlk LSTM katmanı için birim sayısı
units2 = units1*2  # İkinci LSTM katmanı için birim sayısı
units3 = units1  # Yoğun katmanlar için birim sayısı
units3 = units1/2  # Yoğun katmanlar için birim sayısı
units4 = units3/2  # Yoğun katmanlar için birim sayısı
units5 = units4/2  # Yoğun katmanlar için birim sayısı


dropout = 0.2  # Dropout oranı
batch_size = X_train.shape[0] / 8 # Batch boyutu
batch_size = int(batch_size)
activation = 'linear'  # Aktivasyon fonksiyonu
epochs = 50  # Eğitim sayısı

In [10]:
input_shape=(X_train.shape[1], X_train.shape[2])
input_shape

(7, 6)

In [11]:
output_shape = y_train.shape[1]
output_shape

6

In [12]:
model = Sequential()  # Model oluşturma
model.add(LSTM(units1, return_sequences=True, input_shape=input_shape))  # LSTM katmanı


model.add(Dropout(dropout))

model.add(LSTM(units2, return_sequences=True))



model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(LSTM(units1))
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units3))  # Yoğun katman
model.add(Activation(activation))  # Aktivasyon fonksiyonu
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units4))  # Yoğun katman
model.add(Activation(activation))  # Aktivasyon fonksiyonu
model.add(Dropout(dropout))
model.add(BatchNormalization())

model.add(Dense(units5))  # Yoğun katman
model.add(Activation(activation))  # Aktivasyon fonksiyonu
model.add(Dropout(dropout))
model.add(BatchNormalization())





model.add(Dense(output_shape))  # Yoğun katman
model.add(Activation(activation))  # Aktivasyon fonksiyonu


model.compile(optimizer='adam', loss='mean_squared_error' )  # Model derleme



In [13]:
model.summary()  # Model özeti

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 256)            269312    
                                                                 
 dropout (Dropout)           (None, 7, 256)            0         
                                                                 
 lstm_1 (LSTM)               (None, 7, 512)            1574912   
                                                                 
 dropout_1 (Dropout)         (None, 7, 512)            0         
                                                                 
 batch_normalization (Batch  (None, 7, 512)            2048      
 Normalization)                                                  
                                                                 
 lstm_2 (LSTM)               (None, 256)               787456    
                                                        

In [14]:
early_stopping = EarlyStopping(monitor='val_loss',  # İzlenecek metrik
                               patience=5,          # Metrikte iyileşme olmadığı durumda kaç epoch bekleyeceği
                               verbose=1,           # Eğitim durdurulduğunda bilgi vermesi
                               restore_best_weights=True)  # En iyi ağırlıkları geri yükleme
# ReduceLROnPlateau callback'ini ayarlama
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,  # Öğrenme oranını ne kadar azaltacağı
                              patience=3,  # Kaç epoch boyunca iyileşme olmadığında öğrenme oranını azaltacağı
                              min_lr=0.001,  # Öğrenme oranının düşebileceği minimum değer
                              verbose=1)

In [15]:

# Modeli Eğitme ve Tahmin Yapma
history = model.fit(X_train, y_train, 
                    validation_data=(X_test,y_test) , 
                    epochs=epochs, batch_size= batch_size, 
                    verbose=1,  
                    #callbacks=[early_stopping , reduce_lr],
                        )  # Model eğitimi


Epoch 1/50
8/9 [=========================>....] - ETA: 0s - loss: 0.9755

In [ ]:
# Modelin Eğitim ve Test Hata Grafiği
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Eğitim Verisi')
plt.plot(history.history['val_loss'], label='Test Verisi')
plt.title('Model Hatası')
plt.ylabel('Hata')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
model.evaluate(X_test, y_test)  # Test verileri ile modelin performansını ölçme
# bu ne  ? yüksek çıkması iyi mi kötü mü ?


In [ ]:
predictions = model.predict(X_test).squeeze() # Test verisi üzerinde tahmin yapma


In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume'])  # Tahminlerin DataFrame'e dönüştürülmesi
predictions_df

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume'])  # Gerçek değerlerin DataFrame'e dönüştürülmesi
y_test_df

In [ ]:
MSE = mean_squared_error( predictions , y_test)  # Hata ölçümü
MAE = mean_absolute_error( predictions ,y_test)  # Hata ölçümü
R2 = r2_score(predictions ,y_test)  # Hata ölçümü
print('MSE: ', MSE)
print('MAE: ', MAE)
print('R2: ', R2)


In [ ]:
# Sonuçların Değerlendirilmesi
predictions_scaled = scaler.inverse_transform(predictions)  # Tahminlerin ölçeklendirilmesinin tersine çevrilmesi
y_test_scaled = scaler.inverse_transform(y_test)  # Gerçek test verilerinin ölçeklendirilmesinin tersine çevrilmesi


MSE = mean_squared_error( predictions_scaled , y_test_scaled)  # Hata ölçümü
MAE = mean_absolute_error( predictions_scaled ,y_test_scaled)  # Hata ölçümü
R2 = r2_score(predictions_scaled ,y_test_scaled)  # Hata ölçümü
print('MSE: ', MSE)
print('MAE: ', MAE)
print('R2: ', R2)


In [ ]:
predictions_scaled_df = pd.DataFrame(predictions_scaled, columns=['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume'])  # Tahminlerin DataFrame'e dönüştürülmesi
predictions_scaled_df

In [ ]:
y_test_scaled_df = pd.DataFrame(y_test_scaled, columns=['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume'])  # Gerçek değerlerin DataFrame'e dönüştürülmesi
y_test_scaled_df

In [ ]:
def prepare_data_for_forecasting(latest_data, time_step):
    # Son kullanılabilir verileri al ve ölçeklendir
    last_scaled_data = scaler.transform(latest_data)

    # Son verileri model girdisi olarak hazırla
    X = []
    for i in range(len(last_scaled_data) - time_step):
        X.append(last_scaled_data[i:i + time_step, 0:])

    return np.array(X)


In [ ]:
def predict_future_prices(model, prepared_data, days_to_predict):
    predictions = []

    # Son verileri kullanarak gelecek günler için tahmin yap
    current_batch = prepared_data[-1].reshape(1, prepared_data.shape[1], 6)

    for i in range(days_to_predict):
        # Tahmin yap ve sonucu sakla
        current_pred = model.predict(current_batch)[0]
        predictions.append(current_pred)
        
        # Sonraki tahmin için girdiyi güncelle
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

    return scaler.inverse_transform(predictions)


In [ ]:
#btc_data = yf.download('BTC-USD', period='6000d')  # Bitcoin verilerini yfinance ile indirme


In [ ]:
days_to_predict = 7  # Kaç gün sonrası için tahmin yapılacağı

btc_data_son_x = btc_data[-days_to_predict:]


#btc_datadan son x günü sil (göz ile test için)
btc_data = btc_data[:-days_to_predict]


# Son kullanılabilir verileri al (örneğin son 100 gün)
latest_data = btc_data

# Verileri hazırla
prepared_data = prepare_data_for_forecasting(latest_data, time_step=time_step) # Son 100 gündeki verileri kullanarak gelecek 30 gün için tahmin yap

# Gelecek x gün için tahmin yap
future_predictions = predict_future_prices(model, prepared_data, days_to_predict=days_to_predict) # Son 100 gündeki verileri kullanarak gelecek 30 gün için tahmin yapar



In [ ]:
# Tahmin edilen verileri df yap
future_predictions_df = pd.DataFrame(future_predictions, columns=['Open', 'High', 'Low', 'Close' , 'Adj Close', 'Volume'])

#btc_datanın son tarihinden başlayarak 30 gün boyunca tarihleri oluştur
future_predictions_df['Date'] = pd.date_range(start=btc_data.index[-1], periods=len(future_predictions_df), freq='D')

# Tarihi indeks olarak ayarla
future_predictions_df.set_index('Date', inplace=True)
future_predictions_df

In [ ]:
btc_data_son_x